In [1]:
import pyanitools as pyt
import pyaniasetools as aat
import hdnntools as hdt
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import aniensemblestats as ens

import os
import pandas as pd

import time

import seaborn as sns
print(pd.__version__)

#import seaborn as sns
pd.options.display.float_format = '{:.2f}'.format
cm = sns.light_palette("green", as_cmap=True)

/usr/local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


0.20.3


In [2]:
tex_store = '/home/jujuman/Dropbox/CONJCtest_ANI-9.0.5.tex'
xls_store = '/home/jujuman/Dropbox/CONJCtest_ANI-9.0.5.xlsx'
frc_xlsx = '/home/jujuman/Dropbox/CONJCtest_frcErrorperMolecule_ANI-9.0.5.xlsx'
ani_version = 'ANI-2x'

fdir_wb97 = '/home/jujuman/Research/conjugation_check/split_files/data/'

wkdir = '/home/jsmith48/scratch/ANI-2x_retrain/train/ANI-2x_v62318/'
#wkdir = '/home/jujuman/Research/ANI-validation/'
cnstfile = wkdir + 'train0/rHCNOSFCl-5.2R_16-3.8A_a4-8.params'
saefile  = wkdir + 'train0/sae_linfit.dat'
nnfdir   = wkdir + '/train'
Nn = 8

In [3]:
# Define the conformer cross validator class
anicv = aat.anicrossvalidationmolecule(cnstfile,saefile,nnfdir,Nn,gpuid=6)

In [4]:
files = os.listdir(fdir_wb97)

stats1 = dict()
stats4 = dict()
style_F = dict()
Forces = dict({'ANI':[],
               'DFT':[],})

forces = dict({'ANI':{},
               'DFT':{},})


denegy = dict({'ANI':{},
               'DFT':{},})

enegy = dict({'ANI':{},
               'DFT':{},})

for f in files:
    name = f.split('_')[0]
    data_wb97 = hdt.readncdatall(fdir_wb97+f)
    
    start = time.time()
    Eani, Fani, Sani = anicv.compute_stats_multi_molecule(X=data_wb97['coordinates'],S=data_wb97['species'])
    end = time.time()
    comp_time = end - start
    
    Sani = np.std(Eani,axis=0)
    print(Sani.shape)
    Eani = np.mean(Eani,axis=0)
    Fani = np.mean(Fani,axis=0)
    
    Ewb97 = hdt.hatokcal*data_wb97['energies']
    Fwb97 = hdt.hatokcal*data_wb97['forces']
    
    print(name,Ewb97.size,' Time: ', comp_time)
    #print(data['species'])
    #print((Fani-Fdft)[0][12])
    #print(Fani[0][12])
    #print(Fdft[0][12])

    frc_error = dict()
    dF = np.linalg.norm(Fani - Fwb97,axis=2)
    #dF = np.mean(dF,axis=1)
    print(dF.shape)
    for j,F in enumerate(dF):
        Fstore = dict()
        for i,(s,x) in enumerate(zip(data_wb97['species'],F)):
            Fstore[str(i).zfill(3)+' '+s] = x
        #Fstore['Total'] = np.mean(dF,axis=1)
        #print(np.mean(dF,axis=1).shape)
        frc_error[str(j).zfill(3)] = pd.Series(Fstore)
    style_F[name] = pd.DataFrame(frc_error).transpose().style.background_gradient(cmap=cm)
    
    #np.random.shuffle(Fwb97)
    
    Fani = Fani.flatten()
    Fwb97 = Fwb97.flatten()
    
    Forces['ANI'].append(Fani)
    Forces['DFT'].append(Fwb97)

    forces['ANI'].update({name : Fani})
    forces['DFT'].update({name : Fwb97})

    dEani = hdt.calculatedmat(Eani)
    dEwb97 = hdt.calculatedmat(Ewb97)
    
    denegy['ANI'].update({name : dEani})
    denegy['DFT'].update({name : dEwb97})
    
    enegy['ANI'].update({name : Eani})
    enegy['DFT'].update({name : Ewb97})
    
    Na = len(data_wb97['species'])
    
    stats1[name] = pd.Series({'$\mathrm{Atoms}$': int(Na),
                              '$\mathrm{E_{MAE}}$'   : hdt.calculatemeanabserror(Eani,Ewb97),
                              '$\mathrm{E_{RMS}}$'   : hdt.calculaterootmeansqrerror(Eani,Ewb97),
                              '$\mathrm{E_{range}}$'   : np.abs(Ewb97.max()-Ewb97.min()),
                              '$\mathrm{\Delta E_{MAE}}$'  : hdt.calculatemeanabserror(dEani,dEwb97),
                              '$\mathrm{\Delta E_{RMS}}$'  : hdt.calculaterootmeansqrerror(dEani,dEwb97),
                              '$\mathrm{\Delta E_{range}}$'   : np.abs(dEwb97.max()-dEwb97.min()),
                              '$\mathrm{F_{MAE}}$'   : hdt.calculatemeanabserror(Fani,Fwb97),
                              '$\mathrm{F_{RMS}}$'   : hdt.calculaterootmeansqrerror(Fani,Fwb97),
                              '$\mathrm{F_{range}}$'   : np.abs(Fwb97.max()-Fwb97.min()),
                              'time(ms)/mol' : 1000*comp_time/Ewb97.size,
                              })
 
    stats4[name] = pd.Series({'$\mathrm{Atoms}$': int(Na),
                              '$\mathrm{E_{RMS}/ atom}$'   : hdt.calculaterootmeansqrerror(Eani/Na,Ewb97/Na),
                              '$\mathrm{E_{RMS}/ \sqrt{atom}}$'   : hdt.calculaterootmeansqrerror(Eani,Ewb97)/np.sqrt(Na),
                              '$\mathrm{\sigma / \sqrt{atom}}$'   : np.mean(Sani/np.sqrt(Na)),
                              '$\mathrm{\Delta E_{RMS}/ atom}$'  : hdt.calculaterootmeansqrerror(dEani,dEwb97)/Na,
                              '$\mathrm{\Delta E_{RMS}/ \sqrt{atom}}$'  : hdt.calculaterootmeansqrerror(dEani,dEwb97)/np.sqrt(Na),
                              })

for k in Forces.keys():
    Forces[k] = np.concatenate(Forces[k])
    
edat1 = pd.DataFrame(stats1)
edat4 = pd.DataFrame(stats4)
    
#dat4['mean'] = edat4.mean(axis=1)

edat1 = edat1.transpose()
edat4 = edat4.transpose()

FileNotFoundError: [Errno 2] No such file or directory: '/home/jujuman/Research/conjugation_check/split_files/data/'

In [ ]:
#edat = pd.DataFrame(stats).transpose()
pd.set_option('expand_frame_repr', False)
edat1 = edat1.sort_values(['$\mathrm{Atoms}$'])
edat4 = edat4.sort_values(['$\mathrm{Atoms}$']).transpose()
edat4['mean'] = edat4.mean(axis=1)
edat4 = edat4.transpose()
#edat2 = edat2.sort_values(['$\mathrm{Atoms}$'])
#edat3 = edat3.sort_values(['$\mathrm{Atoms}$'])

# Build latex document
header = r'\documentclass{article}'+'\n'+r'\usepackage{booktabs}'+'\n'+r'\usepackage[margin=0.5in]{geometry}'+'\n'+r'\begin{document}'+'\n'

begtab = r'\begin{table}'+'\n'+r'\tiny'+'\n'+r'\centering'+'\n'
endtab = r'\end{table}'+'\n'
enddoc = r'\end{document}'
with open(tex_store, "w") as text_file:
    text_file.write(header)
    text_file.write(begtab+r'\caption{'+ani_version+r'}'+'\n'+edat1.to_latex(escape=False)+endtab)
    text_file.write(begtab+r'\caption{'+'ANI per atom errors'+r'}'+'\n'+edat4.to_latex(escape=False)+endtab)
    text_file.write(enddoc)
    

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(xls_store, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
edat1.to_excel(writer, sheet_name=ani_version)
edat4.to_excel(writer, sheet_name='Per atom errors')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [ ]:
edat1 = edat1.sort_values(['$\mathrm{Atoms}$'])
edat1.style

In [ ]:
edat4.transpose().style

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(frc_xlsx, engine='xlsxwriter')


for k in style_F:
    # Convert the dataframe to an XlsxWriter Excel object.
    style_F[k].to_excel(writer, sheet_name=k+'_'+ani_version)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [ ]:
import aniensemblestats as aes
import matplotlib as mpl

from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
storepath = '/home/jujuman/Dropbox/ANAKIN-ME-RESEARCH/PUBLICATION-ActiveLearning/Figures/force_ANIMD_compare.pdf'

label_size = 20
mpl.rcParams['xtick.labelsize'] = label_size
mpl.rcParams['ytick.labelsize'] = label_size

#'$F_{dft}$' + r' $(kcal \times mol^{-1} \times \AA^{-1})$'

fig, axes = plt.subplots(1, 3, figsize=[30,8], sharex=True, sharey=True, gridspec_kw = {'width_ratios':[1, 1, 1.2]})
cmap = mpl.cm.viridis

bin1 = aes.plot_corr_dist_axes(axes[0], Forces['ANI'], Forces['DFT'], cmap, r'ANI-AL6 Force ($kcal \times mol^{-1} \times \AA^{-1}$)', r'DFT Force ($kcal \times mol^{-1} \times \AA^{-1}$)')
heights = bin1[0]
bin2 = aes.plot_corr_dist_axes(axes[1], Forces['DFTB'], Forces['DFT'], cmap, r'DFTB Force ($kcal \times mol^{-1} \times \AA^{-1}$)', '', 1, heights.max())
bin3 = aes.plot_corr_dist_axes(axes[2], Forces['PM6'], Forces['DFT'], cmap, r'PM6 Force ($kcal \times mol^{-1} \times \AA^{-1}$)', '', 1, heights.max())

cb1 = fig.colorbar(bin2[-1], cmap=cmap)
cb1.set_label('Count', fontsize=20)

#aes.add_inset_histogram(Forces['ANI'], Forces['DFT'], pos=[0.15, 0.636, .07, .235], ylim=[0, 1200], xlim=[-25, 25])
#aes.add_inset_histogram(Forces['DFTB'], Forces['DFT'], pos=[0.41, 0.636, .07, .235], ylim=[0, 1200], xlim=[-25, 25])
#aes.add_inset_histogram(Forces['PM6'], Forces['DFT'], pos=[0.67, 0.636, .07, .235], ylim=[0, 1200], xlim=[-25, 25])
#add_inset_histogram(ax, Xa, Xp)
#add_inset_histogram(ax, Xa, Xp)

plt.show()
#pp = PdfPages(storepath)
#pp.savefig(fig)
#pp.close()

In [ ]:
storepath = '/home/jujuman/Dropbox/ANAKIN-ME-RESEARCH/PUBLICATION-ActiveLearning/Figures/denergy_ANIMD_compare.pdf'

label_size = 20
mpl.rcParams['xtick.labelsize'] = label_size
mpl.rcParams['ytick.labelsize'] = label_size

#'$F_{dft}$' + r' $(kcal \times mol^{-1} \times \AA^{-1})$'

fig, axes = plt.subplots(1, 1, figsize=[10,8])
cmap = mpl.cm.viridis

bin1 = aes.plot_corr_dist_axes(axes, forces['ANI']['Fentanyl'], forces['DFT']['Fentanyl'], cmap, r'ANI-AL6 $\Delta$E ($kcal \times mol^{-1}$)', r'DFT $\Delta$E ($kcal \times mol^{-1}$)')

cb1 = fig.colorbar(bin1[-1], cmap=cmap)
cb1.set_label('Count', fontsize=20)

#aes.add_inset_histogram(Forces['ANI'], Forces['DFT'], pos=[0.15, 0.636, .07, .235], ylim=[0, 1200], xlim=[-25, 25])
#aes.add_inset_histogram(Forces['DFTB'], Forces['DFT'], pos=[0.41, 0.636, .07, .235], ylim=[0, 1200], xlim=[-25, 25])
#aes.add_inset_histogram(Forces['PM6'], Forces['DFT'], pos=[0.67, 0.636, .07, .235], ylim=[0, 1200], xlim=[-25, 25])
#add_inset_histogram(ax, Xa, Xp)
#add_inset_histogram(ax, Xa, Xp)

plt.show()
#pp = PdfPages(storepath)
#pp.savefig(fig)
#pp.close()

In [ ]:
plt.hist(Forces['DFTB'] - Forces['DFT'],bins=100)
plt.hist(Forces['ANI'] - Forces['DFT'],bins=100, alpha=0.5)

plt.show()

In [ ]:
Ed = forces['DFT']['Retinol']
Ea = forces['ANI']['Retinol']

fig = plt.figure(figsize=[8,8])
plt.scatter(Ed, Ea)
plt.plot([Ed.min(), Ed.max()], [Ed.min(), Ed.max()],'--',c='r')
plt.xlim([Ed.min(), Ed.max()])
plt.ylim([Ed.min(), Ed.max()])
plt.xlabel(r"E DFT ($kcal \times mol^{-1}$)", fontsize=18)
plt.ylabel(r"E ani ($kcal \times mol^{-1}$)", fontsize=18)
plt.show()
#pp = PdfPages("/home/jujuman/Dropbox/ANAKIN-ME-RESEARCH/Figures/NEW-11-30-17/fent_ecorr_plot.pdf")
#pp.savefig(fig)
#pp.close()